In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Ge

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
#getting datafiles*.
from pyspark import SparkFiles
url = "https://pdmintelligence-databootcamp-bucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [4]:
#NOTE: used the following pull from postgress to test before pulling here:
#SELECT * FROM information_schema.tables;

#SELECT * FROM vine_table;

#Table appears to be calculated correctly* - pulling columns into vine_df

vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|          0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|                Y|
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|                Y|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|     

In [5]:
#NOTE: Starting deliverable 2**.

In [6]:
# filter for total_votes equal to or greater than 20*.
vote_count = vine_df.filter("total_votes >= 20")
vote_count.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| R4IAIDV5EE84W|          1|            8|         21|   N|                Y|
|R26MB2DA7ROT4P|          1|            2|         21|   N|                Y|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
| RYBZ6CTBOGI0H|          1|            3|         84|   N|     

In [7]:
#Filter on df - retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_df = vote_count.filter("helpful_votes/total_votes>=0.5")
helpful_df.show(1000)

#NOTE: There does not appear to be any instance of vine being equal to 'Y' 
#An iteration of 1000 cases appears to show no instances of vine having the value of 'Y' in this dataset*.


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [8]:
#Filter on helpful_df - create new df taking all rows where reviews vine == 'Y'
#NOTE: issue fixed - was applying show() on the filtered version without the actual creation of the new tables*.
Y_helpful_df=helpful_df.filter(helpful_df["vine"] =="Y")
Y_helpful_df.show()

#NOTE: THERE IS NO 'Y" in this column; all values appear to be 'N'*;


+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [9]:
#Filter on helpful_df - create new df taking all rows where reviews vine == 'N'
#NOTE: issue fixed - was applying show() on the filtered version without the actual creation of the new tables*.
N_helpful_df=helpful_df.filter(helpful_df["vine"] =="N")
N_helpful_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [10]:
#Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
#This instance calculated on all data; no filters for 'y' or 'n' in the files*.
#this is all that is needed since there is no variation on the metric of 'y' or 'n' for the vine column*.
#all values are 'N'*.
FIVE_STARS = helpful_df.filter(helpful_df["star_rating"]== 5)
FIVE_STARS.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RP3K0YBQIVGXZ|          5|           57|         63|   N|                Y|
|R3B5SVW0WVUHRB|          5|           23|         23|   N|                Y|
|R298OT8VO3655T|          5|           22|         25|   N|                Y|
|R22G9KVRZMFBGD|          5|           26|         26|   N|                N|
|R2STEO5WQ426BH|          5|           21|         24|   N|                Y|
| R4UL4LMRE1I8G|          5|           26|         29|   N|     

In [11]:
#counting total 5 stars*.
FIVE_STARS.count()


24673

In [12]:
#counts of paid reviews*. 
Y_helpful_df.count()

0

In [13]:
#counts of nonpaid reviews*.
N_helpful_df.count()


65149

In [16]:
#doing percentages of 5 star scores as well*.
N_FIVE_STAR_HELP_DF = N_helpful_df.filter(helpful_df["star_rating"]== 5)
N_FIVE_STAR_HELP_DF.count()

24673

In [17]:
#doing hte percentages of 5 star scores too*.
Y_FIVE_STAR_HELP_DF = Y_helpful_df.filter(helpful_df["star_rating"]== 5)
Y_FIVE_STAR_HELP_DF.count()

0

In [18]:
#percentage of reviews that were 5 stars*.
#NOTE: comparing 5 star table with share of helpful_df table * they are derived from one another*.
FIVE_STARS.count() / helpful_df.count()


0.37871648068274266

In [19]:
#running the same iteration above but only those with verified purchases*.
#this was computed this way to add more diversity to teh anaylsis since there are no 'paid reviewed' items to assess*.
FIVE_STARS.filter(FIVE_STARS["verified_purchase"] == "Y").count()/helpful_df.filter(helpful_df["verified_purchase"]== "Y").count()



0.3568664526625918

In [20]:
#trying calculations on vine_df*.
FIVE_STARS.count() / vine_df.count()



0.004836237695996211

In [21]:
#trying calculations on vine_df*.
FIVE_STARS.filter(FIVE_STARS["verified_purchase"] == "Y").count()/vine_df.filter(vine_df["verified_purchase"]== "Y").count()


0.0041170488843434196

In [22]:
#doing the data on the columns of paid*.
FIVE_STARS.count() / Y_helpful_df.count()




ZeroDivisionError: ignored

In [23]:
#doing the data on the columns of paid*.
FIVE_STARS.filter(FIVE_STARS["verified_purchase"] == "Y").count()/Y_helpful_df.filter(Y_helpful_df["verified_purchase"]== "Y").count()



ZeroDivisionError: ignored

In [24]:
#doing the data on the columns of UNpaid*.
FIVE_STARS.count() / N_helpful_df.count()


0.37871648068274266

In [25]:
#doing the data on the columns of UNpaid*.
FIVE_STARS.filter(FIVE_STARS["verified_purchase"] == "Y").count()/N_helpful_df.filter(N_helpful_df["verified_purchase"]== "Y").count()

0.3568664526625918

In [28]:
#checking to see if an error was made earlier in the data manipulations*.
#Are we certain that the total counts of paid reviewes are '0' in this dataset*.

Testing_df = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)


In [29]:
Testing_df.filter(Testing_df["vine"]=="Y").count()


0

In [ ]:
#the alternate method of coding the sequence shows again that the value is '0'
#there are no paid reviews in this dataset*.